In [ ]:
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import json

async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        # pages = [f'https://rateyourmusic.com/charts/top/album/all-time/{p}' for p in range(3,11)]
        pages = ['https://rateyourmusic.com/charts/top/album/all-time/4']
        album_info = []
        for p in pages:
            await page.goto(p)
            await page.wait_for_load_state("load")
            for i in range(16):
                await page.wait_for_timeout(500)
                await page.keyboard.down("PageDown")
            pageText = await page.content()
            soup = BeautifulSoup(pageText, 'html.parser')
            
            for album in soup.find_all('div', class_='page_charts_section_charts_item_info'):
                title = album.find('a', class_='page_charts_section_charts_item_link release').text.strip()
                artist = album.find('a', class_='artist').find('span', class_='ui_name_locale').text.strip()
                spotify_link = album.find('a', attrs={'title':'Spotify'})['href']
                primary_genre = album.find('div', class_='page_charts_section_charts_item_genres_primary').text.strip().split('\n')
                secondary_genre = album.find('div', class_='page_charts_section_charts_item_genres_secondary')
                secondary_genre = secondary_genre.text.strip().split('\n') if secondary_genre else []
                descriptors = album.find('div', class_='page_charts_section_charts_item_genre_descriptors').text.strip().split('\n')
                info = {'title': title, 'artist': artist, 'spotify_link': spotify_link, 'primary_genre': primary_genre, 'secondary_genre': secondary_genre, 'descriptors': descriptors}
                album_info.append(info)
        print(album_info)
        with open('./sound-creator/album_info.json', 'wr') as f:
            album_info_r = json.load(f.read()) + album_info
            print(album_info_r)
            # json.dump(album_info, f)
        await browser.close()

loop = asyncio.get_event_loop()
asyncio.set_event_loop(loop)
loop.run_until_complete(main())

In [9]:
from bs4 import BeautifulSoup
import json
import os, sys

files = [f for f in os.listdir('./sound-creator/temp') if(f.endswith('.htm'))]
# album_info = json.load(open('./sound-creator/album_info.json', 'r'))

for filename in files:
    genre = filename.replace('.htm', '').replace('Best albums of all time ', '')
    album_info = []
    content = open(f'./sound-creator/temp/{filename}', 'r').read()
    soup = BeautifulSoup(content, 'html.parser')
    for album in soup.find_all('div', class_='page_charts_section_charts_item_info'):
        try:
            title = album.find('a', class_='page_charts_section_charts_item_link release').text.strip()
            artist = album.find('a', class_='artist').find('span', class_='ui_name_locale').text.strip()
            spotify_link = album.find('a', attrs={'title':'Spotify'})['href']
            
            # primary_genre = album.find('div', class_='page_charts_section_charts_item_genres_primary').text.strip().split('\n')
            # secondary_genre = album.find('div', class_='page_charts_section_charts_item_genres_secondary')
            # secondary_genre = secondary_genre.text.strip().split('\n') if secondary_genre else []
            info = {'title': title, 'artist': artist, 'spotify_link': spotify_link,
                    # 'primary_genre': primary_genre, 'secondary_genre': secondary_genre
                    }
            album_info.append(info)
        except:
            pass
    json.dump(album_info, open(f'./sound-creator/album_info2_{genre}.json', 'w'))
# json.dump(album_info, open('./sound-creator/album_info2.json', 'w'))

In [1]:
from savify import Savify
from savify.types import Type, Format, Quality
from savify.utils import PathHolder
import logging
import json
import os, shutil
from dotenv import load_dotenv

load_dotenv()  

files = [f for f in os.listdir('./sound-creator') if(f.startswith('album_info2_'))]
for filename in files:
    genre = filename.replace('album_info2_', '').replace('.json', '')
    with open(f'./sound-creator/{filename}', 'r') as f:
        album_info = json.load(f)
        for i, info in enumerate(album_info):
            print(f'./sound-creator/albums/{info["artist"]}/{info["title"].replace("/", "-")}')
            if os.path.exists(f'./sound-creator/albums/{genre}/{info["artist"]}/{info["title"].replace("/", "-")}'): 
                print(f'{i} - {info["title"]} by {info["artist"]} already exists')
                # print('')
                # shutil.copytree(f'./sound-creator/output/{info["artist"]}/{info["title"].replace("/", "-")}',
                #                 f'./sound-creator/output_genres/{genre}/{info["artist"]}/{info["title"].replace("/", "-")}')
                continue
            try:
                print(f'{i} - Downloading {info["title"]} by {info["artist"]}')
                path = PathHolder(downloads_path=f'./sound-creator/albums/{genre}/{info["artist"]}/{info["title"].replace("/", "-")}')
                s = Savify(api_credentials=(os.getenv('SPOT_TOKEN'), os.getenv('SPOT_SECRET')), path_holder=path, logger=logging.getLogger("savify"))
                # Spotify URL
                s.download(info['spotify_link'])
            except:
                print('fail')

./sound-creator/albums/Milton Nascimento/Geraes
0 - Downloading Geraes by Milton Nascimento
./sound-creator/albums/Grouper/Dragging a Dead Deer Up a Hill
1 - Downloading Dragging a Dead Deer Up a Hill by Grouper
./sound-creator/albums/Ichiko Aoba
青葉市子/qp
2 - Downloading qp by Ichiko Aoba
青葉市子
fail
./sound-creator/albums/Elliott Smith/Roman Candle
3 - Downloading Roman Candle by Elliott Smith
./sound-creator/albums/John Fahey/Volume 6: Days Have Gone By
4 - Downloading Volume 6: Days Have Gone By by John Fahey
fail
./sound-creator/albums/Phoebe Bridgers/Punisher
5 - Downloading Punisher by Phoebe Bridgers
./sound-creator/albums/John Fahey/America
6 - Downloading America by John Fahey
./sound-creator/albums/Adrianne Lenker/Bright Future
7 - Downloading Bright Future by Adrianne Lenker
./sound-creator/albums/Fleet Foxes/Fleet Foxes
8 - Downloading Fleet Foxes by Fleet Foxes


ERROR: unable to download video data: HTTP Error 403: Forbidden
ERROR: unable to download video data: HTTP Error 403: Forbidden


./sound-creator/albums/Milton Nascimento/Minas
9 - Downloading Minas by Milton Nascimento
./sound-creator/albums/The Mountain Goats/The Sunset Tree
10 - Downloading The Sunset Tree by The Mountain Goats
./sound-creator/albums/Nico/The Marble Index
11 - Downloading The Marble Index by Nico
./sound-creator/albums/Harmonium/Les cinq saisons
12 - Downloading Les cinq saisons by Harmonium
./sound-creator/albums/The Mountain Goats/Tallahassee
13 - Downloading Tallahassee by The Mountain Goats
./sound-creator/albums/Lamp/ランプ幻想 (Lamp Gensō)
14 - Downloading ランプ幻想 (Lamp Gensō) by Lamp
./sound-creator/albums/Zé Ramalho/A peleja do Diabo com o dono do céu
15 - Downloading A peleja do Diabo com o dono do céu by Zé Ramalho
./sound-creator/albums/Joni Mitchell/Ladies of the Canyon
16 - Downloading Ladies of the Canyon by Joni Mitchell
./sound-creator/albums/John Prine/John Prine
17 - Downloading John Prine by John Prine
./sound-creator/albums/Roy Harper/Stormcock
18 - Downloading Stormcock by Roy Ha

ERROR: unable to download video data: HTTP Error 403: Forbidden


./sound-creator/albums/OutKast/Stankonia
3 - Downloading Stankonia by OutKast


...

./sound-creator/albums/The Stooges/Fun House
27 - Downloading Fun House by The Stooges


ERROR: unable to download video data: HTTP Error 403: Forbidden


./sound-creator/albums/Neil Young/After the Gold Rush
28 - Downloading After the Gold Rush by Neil Young
./sound-creator/albums/The Clash/London Calling
29 - Downloading London Calling by The Clash
./sound-creator/albums/The Velvet Underground/White Light - White Heat
30 - Downloading White Light / White Heat by The Velvet Underground
./sound-creator/albums/Radiohead/A Moon Shaped Pool
31 - Downloading A Moon Shaped Pool by Radiohead
./sound-creator/albums/Unwound/Leaves Turn Inside You
32 - Downloading Leaves Turn Inside You by Unwound


ERROR: unable to download video data: HTTP Error 403: Forbidden


./sound-creator/albums/Neil Young //Rust Never Sleeps
33 - Downloading Rust Never Sleeps by Neil Young /
fail
./sound-creator/albums/David Bowie/Hunky Dory
34 - Downloading Hunky Dory by David Bowie
./sound-creator/albums/Car Seat Headrest/Twin Fantasy
35 - Downloading Twin Fantasy by Car Seat Headrest
./sound-creator/albums/Tom Waits/Rain Dogs
36 - Downloading Rain Dogs by Tom Waits
./sound-creator/albums/Neil Young/On the Beach
37 - Downloading On the Beach by Neil Young
./sound-creator/albums/Songs: Ohia/The Magnolia Electric Co
38 - Downloading The Magnolia Electric Co by Songs: Ohia
fail
./sound-creator/albums/Sweet Trip/You Will Never Know Why
39 - Downloading You Will Never Know Why by Sweet Trip


In [2]:
import json
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

albums = pd.DataFrame()
with open('./sound-creator/album_info2.json', 'r') as f:
    album_info = json.load(f)
    genres = []
    for i, info in enumerate(album_info):
        album_info[i]['genres'] = info['primary_genre'] + info['secondary_genre']
        del album_info[i]['primary_genre']
        del album_info[i]['secondary_genre']
        # del album_info[i]['descriptors']
        del album_info[i]['spotify_link']
        genres += album_info[i]['genres']
    
    genres = list(set(genres))
    albums = pd.DataFrame(album_info)
    for genre in genres:
        albums[genre] = albums['genres'].apply(lambda x: genre in x)
        albums[genre] = albums[genre].astype(int)
    albums.drop(columns=['genres'], inplace=True)

albums.index = albums['artist'] + ' - ' + albums['title']
albums.drop(columns=['artist', 'title'], inplace=True)
albums.to_parquet('./sound-creator/albums_classes.parquet')

c:\Users\luish\miniconda3\envs\sound-s\lib\site-packages\ipykernel_launcher.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [49]:
import pandas as pd
import numpy as np
import csv

genres = [l.upper().split(' ') for l in list(pd.read_parquet('./sound-creator/albums_classes.parquet').columns)]
genres = [x for xs in genres for x in xs]

genres = pd.DataFrame(genres).value_counts()
genres.to_csv('./sound-creator/genres.csv')